# Cassandra

## Conexión con Cassandra y borrado de datos

In [1]:
%load_ext cql

In [2]:
%%cql
DROP KEYSPACE black;

'No results.'

In [3]:
%%cql
CREATE KEYSPACE black 
WITH replication = {'class':'SimpleStrategy', 'replication_factor': 1};

'No results.'

In [4]:
%cql USE black;

'No results.'

## Creacción de las tablas

En **Cassandra** vamos a crear 2 tablas principales, una donde almacenamos los movimientos en formato desnormalizado, y otra que nos servirá para almacenar los importes acumulados por cliente.

También tenemos una tabla auxiliar, creada gracias a la funcionalidad que nos da la base de datos de VISTAS MATERIALIZADAS, que vamos a utilizar para facilitar otros patrones distintos de acceso a la información.

En la tabla que almacena los movimientos, como no tenemos una PK clara, vamos a utilizar un tipo UUID (marca temporal única) donde guardaremos un TIMESTAMP con la hora de creación del registro

<br><br> 

<img src="images/Modelo%20Cassandra.png",width=800,height=500>

<br><br> 

In [5]:
%%cql 
CREATE TABLE acum_movimientos_nombre (
    nombre          text,
    importe         counter,
    PRIMARY KEY (nombre)
)

'No results.'

In [6]:
%%cql 
CREATE TABLE movimientos (
    uid             uuid,
    fecha           date,
    hora            int,
    minuto          int,
    importe         decimal,
    comercio        text,
    actividad_completa text,
    actividad       text,
    nombre          text,
    funcion         text,
    organizacion    text,
    PRIMARY KEY ((nombre), importe, fecha, hora, minuto)
)
WITH CLUSTERING ORDER BY (importe DESC);

'No results.'

En la vista materializada no es necesario insertar datos ya que se ocupa la propia base de datos

In [7]:
%%cql
CREATE MATERIALIZED VIEW vm_movimientos_by_actividad AS
   SELECT * FROM movimientos
   WHERE actividad IS NOT NULL 
         and importe IS NOT NULL 
         and fecha IS NOT NULL
         and hora IS NOT NULL 
         and minuto IS NOT NULL 
         and nombre IS NOT NULL
   PRIMARY KEY (actividad, importe, fecha,hora,minuto,nombre)
   WITH CLUSTERING ORDER BY (importe desc)

'No results.'

## Carga de datos en pandas

Cargamos los datos en formato Excel y los desnormalizamos ...

In [8]:
import pandas as pd
df_mov = pd.read_excel("../../data/black.xlsx", sheetname= "Movimientos")
df_miembros = pd.read_excel("../../data/black.xlsx", sheetname= "Miembros")
df = pd.merge(df_mov, df_miembros, on = ['id_miembro'], how = 'inner')
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 7624 entries, 0 to 7623
Data columns (total 11 columns):
id_miembro            7624 non-null int64
fecha                 7624 non-null datetime64[ns]
minuto                7624 non-null int64
hora                  7624 non-null int64
importe               7624 non-null float64
comercio              6889 non-null object
actividad_completa    7623 non-null object
actividad             7623 non-null object
nombre                7624 non-null object
funcion               7624 non-null object
organizacion          6041 non-null object
dtypes: datetime64[ns](1), float64(1), int64(3), object(6)
memory usage: 714.8+ KB


### Carga de datos en Cassandra

In [9]:
from cassandra.cluster import Cluster, BatchStatement, ConsistencyLevel
cluster = Cluster()
session = cluster.connect('black')

A destacar varios aspectos:
- El tipo de las fechas. Hay que dejarlo con el tipo **Date** de Python para no tener problemas (El tipo de la columna es también Date)
- Los nulos en Pandas son del tipo NumPy.nan, por lo hay convertirlos a None para que la inserción en Cassandra sea correcta
- En la tabla de importes acumulados guardamos el importe en un dato de tipo **counter**, que tiene un tipo entero, por lo que multiplicamos el dato por 100 para poder almacenarlo correctamete

In [10]:
df.ix[df.comercio.isnull(), 'comercio'] = None
df.ix[df.organizacion.isnull(), 'organizacion'] = None
df.ix[df.actividad.isnull(), 'actividad'] = None
df.ix[df.actividad_completa.isnull(), 'actividad_completa'] = None

In [11]:
import dateutil

def insert_movientos(df):
    
    sql_insert = """
INSERT INTO movimientos (
uid,
importe,
actividad,
fecha,
actividad_completa,
comercio,
funcion,
hora,
minuto,
organizacion,
nombre
) VALUES (now(), %s, %s, %s, %s, %s, %s, %s, %s, %s, %s)
"""

    for index in df.index:
        fecha = df.ix[index, 'fecha']
        
        data = [
            df.ix[index, "importe"],
            df.ix[index, "actividad"],
            fecha.to_datetime().date(),
            df.ix[index, "actividad_completa"],
            df.ix[index, "comercio"],
            df.ix[index, "funcion"],
            df.ix[index, "hora"],
            df.ix[index, "minuto"],
            df.ix[index, "organizacion"],
            df.ix[index, "nombre"]
        ]
        
        session.execute(sql_insert, data)
        
        importe_int = int(round(df.ix[index, "importe"] * 100))        
        session.execute("UPDATE acum_movimientos_nombre SET importe = importe + %s WHERE nombre = %s", 
                        [importe_int, 
                         df.ix[index, "nombre"]]
        )

Ya podemos insertar los datos ...

In [12]:
insert_movientos(df)

## Consulta de datos

Función de utilidad que realiza un query en Cassandra y devuelve un DataFrame de Pandas

In [13]:
def execute_query(sql):
    rows = session.execute(sql)
    return pd.DataFrame(list(rows))

### Los 10 movimientos mas caros

Para resolver esta query con la base de datos tendríamos que tener un PARTITION KEY única para todos los registros

In [14]:
sql = """
SELECT nombre, fecha, actividad_completa, importe
FROM MOVIMIENTOS
"""
df = execute_query(sql)

In [15]:
df.sort_values('importe', ascending = False).head(10)

,nombre,fecha,actividad_completa,importe
1573,Ricardo Romero de Tejada y Picatoste,2007-11-25,AGENCIAS BANCARIAS(ANTICIPO VENTANILLA),11930.0
5418,Carlos Vela García,2006-11-01,VIAJES MARSANS-INTERNACIONAL EXPRESSO,9825.0
4731,Ildefonso José Sánchez Barcoj,2009-03-30,EL CORTE INGLES,9804.15
2775,Carmen Contreras Gómez,2010-08-15,"HOTELES 4 Y 5 ESTRELLAS,BALNEARIOS,CAMPI",9076.76
778,Enrique de la Torre Martínez,2006-11-29,AGENCIAS BANCARIAS(ANTICIPO VENTANILLA),8000.0
4732,Ildefonso José Sánchez Barcoj,2005-07-19,AGENCIAS BANCARIAS(ANTICIPO VENTANILLA),7200.0
4454,Miguel Blesa de la Parra,2007-08-02,EL CORTE INGLES,6990.87
3666,María Elena Gil García,2004-03-30,JOYERIAS Y RELOJERIAS,6905.72
3667,María Elena Gil García,2003-08-28,JOYERIAS Y RELOJERIAS,6000.0
2412,Mariano Pérez Claver,2004-10-18,AGENCIAS DE VIAJES,6000.0


### Los movimientos de una persona concreta

Se resuelve la query con la base de datos, ya que la PARTITION KEY es el campo **nombre**, y los registros están ordenados por importe

In [16]:
sql = """
select nombre, fecha, actividad_completa, importe
from movimientos
where nombre = 'Javier de Miguel Sánchez'
limit 10
"""
execute_query(sql)

,nombre,fecha,actividad_completa,importe
0,Javier de Miguel Sánchez,2008-06-12,"ELECTRODOMESTICOS,EQUIPOS ELECTRICOS",572.0
1,Javier de Miguel Sánchez,2006-05-01,"HOTELES,MOTELES,BALNEARIOS,CAMPINGS REST",571.65
2,Javier de Miguel Sánchez,2003-09-30,AUTOM.Y MOTOCICLETAS ( VENTAS Y REPARAC),417.88
3,Javier de Miguel Sánchez,2010-01-12,"MATERIALES CONSTRUCCION,FONTANERIA,SANEA",383.0
4,Javier de Miguel Sánchez,2006-12-08,EL CORTE INGLES,304.66
5,Javier de Miguel Sánchez,2008-03-16,"HOTELES 4 Y 5 ESTRELLAS,BALNEARIOS,CAMPI",287.07
6,Javier de Miguel Sánchez,2009-06-19,HIPERCOR SUPERMERCADOS EL CORTE INGLES,283.0
7,Javier de Miguel Sánchez,2009-12-22,"MATERIALES CONSTRUCCION,FONTANERIA,SANEA",272.85
8,Javier de Miguel Sánchez,2003-08-13,"HOTELES,MOTELES,BALNEARIOS,CAMPINGS REST",268.0
9,Javier de Miguel Sánchez,2009-08-16,PARADORES NACIONALES,250.38


### Las 10 personas que mas han gastado

Esta consulta se pueder resolver mediante el acumulado que habíamos creado ad-doc. Observa que el importe es entero, por lo que hay que dividirlo por 100 para convertirlo a decimal

In [17]:
sql = """
select *
from acum_movimientos_nombre
"""
df = execute_query(sql)

In [18]:
df.assign(importe = df.importe / 100).sort_values('importe', ascending = False).head(10)

,nombre,importe
57,Ildefonso José Sánchez Barcoj,66444.15
72,José Antonio Moral Santín,40356.29
62,Carlos Vela García,37639.57
55,Miguel Blesa de la Parra,37608.99
11,Enrique de la Torre Martínez,37472.01
67,Matías Amat Roca,36590.68
33,Maria Mercedes de la Merced Monge,36086.89
21,Ricardo Romero de Tejada y Picatoste,35901.41
73,Ricardo Morado Iglesias,35574.91
4,Ramón Ferraz Ricarte,35136.97


### Los 10 movimientos mas caros por actividad

Esta consulta se pueder resolver mediante la vista materializa que hemos creado ad-hoc para resolver esta consulta

In [19]:
sql = """
select nombre, fecha, actividad, actividad_completa, importe
from vm_movimientos_by_actividad
where actividad = 'HOGAR'
limit 10
"""
execute_query(sql)

,nombre,fecha,actividad,actividad_completa,importe
0,Jesús Pedroche Nieto,2009-05-07,HOGAR,"ELECTRODOMESTICOS,EQUIPOS ELECTRICOS",3794.0
1,Jesús Pedroche Nieto,2009-07-05,HOGAR,"ELECTRODOMESTICOS,EQUIPOS ELECTRICOS",3512.0
2,Rodrigo de Rato Figaredo,2010-02-05,HOGAR,"FERRETERIA,BRICOLAJE,MENAJE DEL HOGAR",1392.03
3,Rafael Spottorno Díaz Caro,2003-08-10,HOGAR,MIRO ESTABLECIMIENTOS,1198.0
4,Jesús Pedroche Nieto,2009-01-02,HOGAR,"MUEBLES,ANTIGUEDADES Y GALERIAS DE ARTE",1000.41
5,Alejandro Couceiro Ojeda,2005-07-04,HOGAR,"ELECTRODOMESTICOS,EQUIPOS ELECTRICOS",998.0
6,Mariano Pérez Claver,2007-03-16,HOGAR,FLORES Y PLANTAS,864.29
7,Alejandro Couceiro Ojeda,2005-06-15,HOGAR,"MUEBLES,ANTIGUEDADES Y GALERIAS DE ARTE",850.0
8,María Carmen Cafranga Cavestany,2006-07-31,HOGAR,"MUEBLES,ANTIGUEDADES Y GALERIAS DE ARTE",850.0
9,Rafael Darío Fernández Yruegas Moro,2006-06-15,HOGAR,"FERRETERIA,BRICOLAJE,MENAJE DEL HOGAR",660.0
